<a href="https://colab.research.google.com/github/yjkno1/AI_TEAM/blob/main/IsolationForest_0329.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**필요파일 Import**


In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import IsolationForest

import ftplib
import os
import pickle
from os import unlink

import plotly.express as px

# Google Drive 에서 파일 가져올때 사용
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)


**Dataframe 생성**

In [118]:
# 위와같이 파일을 읽을때 date 컬럼의 타입을 지정할수 있고
# df['srvtime'] = pd.to_datetime(df['srvtime]) 이런형태로 나중에 변경 할수도 있다.

#csv_file = "http://yjkno1.synology.me:8081/file/ROUGHING_MILL_STAND10_ACTUAL_CURRENT_sample.csv"

csv_file = "http://yjkno1.synology.me:8081/file/TOP_9_TAG.csv"

df = pd.read_csv(csv_file, \
                 parse_dates=['srvtime'], \
                 infer_datetime_format=True)

df.describe()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7627651 entries, 0 to 7627650
Data columns (total 3 columns):
 #   Column   Dtype                                
---  ------   -----                                
 0   tag      object                               
 1   srvtime  datetime64[ns, pytz.FixedOffset(540)]
 2   value    float64                              
dtypes: datetime64[ns, pytz.FixedOffset(540)](1), float64(1), object(1)
memory usage: 174.6+ MB


**index 를 srvtime으로 재설정 하고 date의 분으로 평균값 구하여 재구성 한다.**


In [134]:
df['tag'] = df['tag'].astype(str)

#print(df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_SPEED_RPM'].count())

rpmData = df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_SPEED_RPM'] # 669503
currentData = df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_CURRENT'] # 1047052
torqueData = df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_TORQUE'] # 1015070

rpmData = rpmData[(rpmData['srvtime'] >='2021-03-09') & (rpmData['srvtime'] < '2021-03-10')]
currentData = currentData[(currentData['srvtime'] >='2021-03-09') & (currentData['srvtime'] < '2021-03-10')]
torqueData = torqueData[(torqueData['srvtime'] >='2021-03-09') & (torqueData['srvtime'] < '2021-03-10')]


rpmData=rpmData.set_index('srvtime').resample("1S").mean().reset_index()
currentData=currentData.set_index('srvtime').resample("1S").mean().reset_index()
torqueData=torqueData.set_index('srvtime').resample("1S").mean().reset_index()

# # 평균값의 소숫점을 1자리로 변경
rpmData['value']=round(rpmData['value'], 1)
currentData['value']=round(currentData['value'], 1)
torqueData['value']=round(torqueData['value'], 1)

rpmData['value'].fillna(rpmData['value'].mean(), inplace = True)
currentData['value'].fillna(currentData['value'].mean(), inplace = True)
torqueData['value'].fillna(torqueData['value'].mean(), inplace = True)

print(rpmData.count())
print(currentData.count())
print(torqueData.count())

srvtime    86400
value      86400
dtype: int64
srvtime    86400
value      86400
dtype: int64
srvtime    86400
value      86400
dtype: int64


In [136]:
x_train = rpmData

x_train['rpm'] = rpmData['value']
x_train['current'] = currentData['value']
x_train['torque'] = torqueData['value']

x_train

,srvtime,value,rpm,current,torque
0,2021-03-09 00:00:00+09:00,449.6,449.6,250.5,20.3
1,2021-03-09 00:00:01+09:00,448.3,448.3,252.9,20.6
2,2021-03-09 00:00:02+09:00,453.0,453.0,219.5,2.0
3,2021-03-09 00:00:03+09:00,451.7,451.7,227.6,2.2
4,2021-03-09 00:00:04+09:00,450.0,450.0,214.1,2.4
...,...,...,...,...,...
86395,2021-03-09 23:59:55+09:00,460.6,460.6,268.4,26.0
86396,2021-03-09 23:59:56+09:00,460.7,460.7,258.7,25.6
86397,2021-03-09 23:59:57+09:00,459.6,459.6,259.5,25.5
86398,2021-03-09 23:59:58+09:00,459.1,459.1,268.0,25.8


**Model 만들기 함수**

In [111]:
def isolation_forest_model(state_seed=20, n_estimators=100, max_samples=128, contamination=float(0.004)):
  state_value = np.random.RandomState(state_seed)
  return IsolationForest(n_estimators=100, max_samples=128, contamination=float(0.004), random_state=state_value)


**Model 생성**

In [112]:
isol = isolation_forest_model()

isol

IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.004,
                max_features=1.0, max_samples=128, n_estimators=100,
                n_jobs=None,
                random_state=RandomState(MT19937) at 0x7F432372EAF0, verbose=0,
                warm_start=False)

In [138]:
isol.fit(x_train[['rpm', 'current', 'torque']])


IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.004,
                max_features=1.0, max_samples=128, n_estimators=100,
                n_jobs=None,
                random_state=RandomState(MT19937) at 0x7F432372EAF0, verbose=0,
                warm_start=False)

**Model에 Fit할 데이터 전처리**

In [142]:
x_rpmData = df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_SPEED_RPM'] # 669503
x_currentData = df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_CURRENT'] # 1047052
x_torqueData = df[df['tag'] == 'SRL_2.2YH70.ROUGHING_MILL_STAND10_ACTUAL_TORQUE'] # 1015070

x_rpmData = x_rpmData[x_rpmData['srvtime'] >='2021-03-09']
x_currentData = x_currentData[x_currentData['srvtime'] >='2021-03-09']
x_torqueData = x_torqueData[x_torqueData['srvtime'] >='2021-03-09']

x_rpmData=x_rpmData.set_index('srvtime').resample("1S").mean().reset_index()
x_currentData=x_currentData.set_index('srvtime').resample("1S").mean().reset_index()
x_torqueData=x_torqueData.set_index('srvtime').resample("1S").mean().reset_index()

# # 평균값의 소숫점을 1자리로 변경
x_rpmData['value']=round(x_rpmData['value'], 1)
x_currentData['value']=round(x_currentData['value'], 1)
x_torqueData['value']=round(x_torqueData['value'], 1)

x_rpmData['value'].fillna(x_rpmData['value'].mean(), inplace = True)
x_currentData['value'].fillna(x_currentData['value'].mean(), inplace = True)
x_torqueData['value'].fillna(x_torqueData['value'].mean(), inplace = True)

print(x_rpmData.count())
print(x_currentData.count())
print(x_torqueData.count())

srvtime    141702
value      141702
dtype: int64
srvtime    141702
value      141702
dtype: int64
srvtime    141702
value      141702
dtype: int64


In [144]:
x_test = x_rpmData

x_test['rpm'] = x_rpmData['value']
x_test['current'] = x_currentData['value']
x_test['torque'] = x_torqueData['value']

x_test

,srvtime,value,rpm,current,torque
0,2021-03-09 00:00:00+09:00,449.6,449.6,250.5,20.3
1,2021-03-09 00:00:01+09:00,448.3,448.3,252.9,20.6
2,2021-03-09 00:00:02+09:00,453.0,453.0,219.5,2.0
3,2021-03-09 00:00:03+09:00,451.7,451.7,227.6,2.2
4,2021-03-09 00:00:04+09:00,450.0,450.0,214.1,2.4
...,...,...,...,...,...
141697,2021-03-10 15:21:37+09:00,419.8,419.8,273.4,28.5
141698,2021-03-10 15:21:38+09:00,420.1,420.1,257.8,28.3
141699,2021-03-10 15:21:39+09:00,419.0,419.0,282.0,28.4
141700,2021-03-10 15:21:40+09:00,418.9,418.9,264.4,28.3


In [145]:
series = isol.predict(x_test[['rpm', 'current', 'torque']])

series

x_test['outliers']=pd.Series(series).apply(lambda x: 'yes' if (x == -1) else 'no' )
print(x_test)

# 이상데이터 건수
print(x_test[x_test.outliers=='yes'].count())

# 정상데이터 건수
print(x_test[x_test.outliers=='no'].count())

                         srvtime  value    rpm  current  torque outliers
0      2021-03-09 00:00:00+09:00  449.6  449.6    250.5    20.3       no
1      2021-03-09 00:00:01+09:00  448.3  448.3    252.9    20.6       no
2      2021-03-09 00:00:02+09:00  453.0  453.0    219.5     2.0       no
3      2021-03-09 00:00:03+09:00  451.7  451.7    227.6     2.2       no
4      2021-03-09 00:00:04+09:00  450.0  450.0    214.1     2.4       no
...                          ...    ...    ...      ...     ...      ...
141697 2021-03-10 15:21:37+09:00  419.8  419.8    273.4    28.5       no
141698 2021-03-10 15:21:38+09:00  420.1  420.1    257.8    28.3       no
141699 2021-03-10 15:21:39+09:00  419.0  419.0    282.0    28.4       no
141700 2021-03-10 15:21:40+09:00  418.9  418.9    264.4    28.3       no
141701 2021-03-10 15:21:41+09:00  419.8  419.8    260.9    28.3       no

[141702 rows x 6 columns]
srvtime     1866
value       1866
rpm         1866
current     1866
torque      1866
outliers    

**Scatter 그래프 API를 사용하여 데이터 시각화**


In [147]:
#fig = px.scatter(df.reset_index(), x='srvtime', y='value', color='outliers', hover_data=['weekday', 'hour'], title='Anomaly Data')
fig = px.scatter(x_test.reset_index(), x=['srvtime'], y=['rpm', 'current', 'torque'], color='outliers', hover_data=x_test, title='Anomaly Data')

# 범위를 선택해서 slider로 조절함.(범위 슬라이더)
fig.update_xaxes(
    rangeslider_visible=True,
)
fig.show()

ValueError: ignored